In [1]:
from isaac_victor_envs.utils import get_assets_dir
from isaac_victor_envs.tasks.allegro import AllegroScrewdriverTurningEnv
# from isaac_victor_envs.tasks.allegro_ros import RosAllegroValveTurningEnv

import numpy as np
import pickle

import torch
import time
import copy
import yaml
import pathlib
from functools import partial
import sys

import pytorch_volumetric as pv
import pytorch_kinematics as pk
import pytorch_kinematics.transforms as tf
from torch.func import vmap, jacrev, hessian, jacfwd
# import pytorch3d.transforms as tf

import matplotlib.pyplot as plt
from ccai.utils.allegro_utils import *
# from allegro_valve_roll import AllegroValveTurning, AllegroContactProblem, PositionControlConstrainedSVGDMPC, \
#    add_trajectories, add_trajectories_hardware

from ccai.allegro_contact import AllegroManipulationProblem, PositionControlConstrainedSVGDMPC, add_trajectories, \
    add_trajectories_hardware
from ccai.allegro_screwdriver_problem_diffusion import AllegroScrewdriverDiff
from scipy.spatial.transform import Rotation as R

# from ccai.mpc.ipopt import IpoptMPC
# from ccai.problem import IpoptProblem
from ccai.models.trajectory_samplers import TrajectorySampler

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

Importing module 'gym_38' (/home/abhinav/Downloads/IsaacGym_Preview_4_Package/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/abhinav/Downloads/IsaacGym_Preview_4_Package/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.3.1+cu118
Device count 1
/home/abhinav/Downloads/IsaacGym_Preview_4_Package/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/abhinav/.cache/torch_extensions/py38_cu118 as PyTorch extensions root...
Emitting ninja build file /home/abhinav/.cache/torch_extensions/py38_cu118/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
obj_dof = 3
# config = yaml.safe_load(pathlib.Path(f'../examples/config/{sys.argv[1]}.yaml').read_text())
config = yaml.safe_load(pathlib.Path(f'../examples/config/allegro_screwdriver_proj_diff_init.yaml').read_text())
config['visualize'] = False
from tqdm import tqdm

if config['mode'] == 'hardware':
    env = RosAllegroValveTurningEnv(1, control_mode='joint_impedance',
                                    use_cartesian_controller=False,
                                    viewer=True,
                                    steps_per_action=60,
                                    friction_coefficient=1.0,
                                    device=config['sim_device'],
                                    valve=config['object_type'],
                                    video_save_path=img_save_dir,
                                    joint_stiffness=config['kp'],
                                    fingers=config['fingers'],
                                    )
else:
    if not config['visualize']:
        img_save_dir = None

    env = AllegroScrewdriverTurningEnv(1, control_mode='joint_impedance',
                                        use_cartesian_controller=False,
                                        viewer=config['visualize'],
                                        steps_per_action=60,
                                        friction_coefficient=config['friction_coefficient'] * 1.05,
                                        # friction_coefficient=1.0,  # DEBUG ONLY, set the friction very high
                                        device=config['sim_device'],
                                        video_save_path=img_save_dir,
                                        joint_stiffness=config['kp'],
                                        fingers=config['fingers'],
                                        )

sim, gym, viewer = env.get_sim()

state = env.get_state()
# try:
#     while True:
#         start = env.get_state()['q'][:, :-1]
#         env.step(start)
#         print('waiting for you to finish camera adjustment, ctrl-c when done')
#         time.sleep(0.1)
# except KeyboardInterrupt:
#     pass

sim_env = None
ros_copy_node = None
if config['mode'] == 'hardware':
    sim_env = env
    from hardware.hardware_env import HardwareEnv

    env = HardwareEnv(sim_env.default_dof_pos[:, :16], finger_list=['index', 'thumb'], kp=config['kp'])
    env.world_trans = sim_env.world_trans
    env.joint_stiffness = sim_env.joint_stiffness
    env.device = sim_env.device
    env.valve_pose = sim_env.valve_pose
elif config['mode'] == 'hardware_copy':
    from hardware.hardware_env import RosNode

    ros_copy_node = RosNode()

results = {}

# set up the kinematic chain
asset = f'{get_assets_dir()}/xela_models/allegro_hand_right.urdf'
ee_names = {
    'index': 'allegro_hand_hitosashi_finger_finger_0_aftc_base_link',
    'middle': 'allegro_hand_naka_finger_finger_1_aftc_base_link',
    'ring': 'allegro_hand_kusuri_finger_finger_2_aftc_base_link',
    'thumb': 'allegro_hand_oya_finger_3_aftc_base_link',
}
config['ee_names'] = ee_names
config['obj_dof'] = 3

screwdriver_asset = f'{get_assets_dir()}/screwdriver/screwdriver.urdf'

chain = pk.build_chain_from_urdf(open(asset).read())
screwdriver_chain = pk.build_chain_from_urdf(open(screwdriver_asset).read())
frame_indices = [chain.frame_to_idx[ee_names[finger]] for finger in config['fingers']]  # combined chain
frame_indices = torch.tensor(frame_indices)
state2ee_pos = partial(state2ee_pos, fingers=config['fingers'], chain=chain, frame_indices=frame_indices,
                        world_trans=env.world_trans)

forward_kinematics = partial(chain.forward_kinematics,
                                frame_indices=frame_indices)  # full_to= _partial_state = partial(full_to_partial_state, fingers=config['fingers'])
partial_to_full_state = partial(partial_to_full_state, fingers=config['fingers'])

controller = 'csvgd'
goal = - 0.5 * torch.tensor([0, 0, np.pi])
# set up params
params = config.copy()
params.pop('controllers')
params.update(config['controllers'][controller])
params['controller'] = controller
params['valve_goal'] = goal.to(device=params['device'])
params['chain'] = chain.to(device=params['device'])
object_location = torch.tensor([0, 0, 1.205]).to(
    params['device'])  # TODO: confirm if this is the correct location
params['object_location'] = object_location

num_fingers = len(params['fingers'])
state = env.get_state()
start = state['q'].reshape(4 * num_fingers + 4).to(device=params['device'])
if 'csvgd' in params['controller']:
    # index finger is used for stability
    if 'index' in params['fingers']:
        fingers = params['fingers']
    else:
        fingers = ['index'] + params['fingers']

pregrasp_problem = AllegroScrewdriverDiff(
    start=start[:4 * num_fingers + obj_dof],
    goal=params['valve_goal'] * 0,
    T=params['T'],
    chain=params['chain'],
    device=params['device'],
    object_asset_pos=env.table_pose,
    object_location=params['object_location'],
    object_type=params['object_type'],
    world_trans=env.world_trans,
    regrasp_fingers=fingers,
    contact_fingers=[],
    obj_dof=obj_dof,
    obj_joint_dim=1,
    optimize_force=params['optimize_force'],
)
# finger gate index
index_regrasp_problem = AllegroScrewdriverDiff(
    start=start[:4 * num_fingers + obj_dof],
    goal=params['valve_goal'] * 0,
    T=params['T'],
    chain=params['chain'],
    device=params['device'],
    object_asset_pos=env.table_pose,
    object_location=params['object_location'],
    object_type=params['object_type'],
    world_trans=env.world_trans,
    regrasp_fingers=['index'],
    contact_fingers=['middle', 'thumb'],
    obj_dof=obj_dof,
    obj_joint_dim=1,
    optimize_force=params['optimize_force'],
    default_dof_pos=env.default_dof_pos[:, :16]
)
thumb_and_middle_regrasp_problem = AllegroScrewdriverDiff(
    start=start[:4 * num_fingers + obj_dof],
    goal=params['valve_goal'] * 0,
    T=params['T'],
    chain=params['chain'],
    device=params['device'],
    object_asset_pos=env.table_pose,
    object_location=params['object_location'],
    object_type=params['object_type'],
    world_trans=env.world_trans,
    contact_fingers=['index'],
    regrasp_fingers=['middle', 'thumb'],
    obj_dof=obj_dof,
    obj_joint_dim=1,
    optimize_force=params['optimize_force'],
    default_dof_pos=env.default_dof_pos[:, :16]
)
turn_problem = AllegroScrewdriverDiff(
    start=start[:4 * num_fingers + obj_dof],
    goal=params['valve_goal'] * 0,
    T=params['T'],
    chain=params['chain'],
    device=params['device'],
    object_asset_pos=env.table_pose,
    object_location=params['object_location'],
    object_type=params['object_type'],
    world_trans=env.world_trans,
    contact_fingers=['index', 'middle', 'thumb'],
    obj_dof=obj_dof,
    obj_joint_dim=1,
    optimize_force=params['optimize_force'],
    default_dof_pos=env.default_dof_pos[:, :16]
)
contact_mode_dict = {0: 'pregrasp', 2: 'index', 1: 'thumb_middle', 3: 'turn'}
t = 15
# with open(data_path / f'constraint_violations.p', 'wb') as f:
#     pickle.dump(constraint_violations_all, f)


Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:0
GPU Pipeline: enabled
Using VHACD cache directory '/home/abhinav/.isaacgym/vhacd'
Found existing convex decomposition for mesh '/home/abhinav/Documents/git_packages/isaacgym-arm-envs/isaac_victor_envs/assets/xela_models/mesh/allegro/base_ns.stl'
Found existing convex decomposition for mesh '/home/abhinav/Documents/git_packages/isaacgym-arm-envs/isaac_victor_envs/assets/xela_models/mesh/allegro/link_1.0.stl'
Found existing convex decomposition for mesh '/home/abhinav/Documents/git_packages/isaacgym-arm-envs/isaac_victor_envs/assets/xela_models/mesh/ft_c.stl'


In [3]:
def calc_info(c_state, to_eval, info_list, pregrasp_problem, index_regrasp_problem, thumb_and_middle_regrasp_problem, turn_problem, plans_or_inits):
    if c_state == 'pregrasp':
        # Call _con_eq and _con_ineq to get constraint violation
        pregrasp_problem._preprocess(to_eval, projected_diffusion=False)
        g, _, _ = pregrasp_problem._con_eq(to_eval[..., :27], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)
        h, _, _ = pregrasp_problem._con_ineq(to_eval[..., :27], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)

        info = {'g': g, 'h': h, 'c_state': 'pregrasp'}
        info_list.append(info)
    elif c_state == 'index':
        index_regrasp_problem._preprocess(to_eval, projected_diffusion=False)
        # Need to rearrange (index, middle, thumb)
        if plans_or_inits == 'plans':
            inds = torch.cat((torch.arange(27), torch.arange(30, 36)))
        else:
            inds = torch.arange(33)
        g, _, _ = index_regrasp_problem._con_eq(to_eval[..., inds], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)
        h, _, _ = index_regrasp_problem._con_ineq(to_eval[..., inds], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)

        info = {'g': g, 'h': h, 'c_state': 'index'}
        info_list.append(info)
    elif c_state == 'thumb_middle':
        thumb_and_middle_regrasp_problem._preprocess(to_eval, projected_diffusion=False)
        g, _, _ = thumb_and_middle_regrasp_problem._con_eq(to_eval[..., :30], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)
        h, _, _ = thumb_and_middle_regrasp_problem._con_ineq(to_eval[..., :30], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)

        info = {'g': g, 'h': h, 'c_state': 'thumb_middle'}
        info_list.append(info)
    elif c_state == 'turn':
        turn_problem._preprocess(to_eval, projected_diffusion=False)
        g, _, _ = turn_problem._con_eq(to_eval[..., :], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)
        h, _, _ = turn_problem._con_ineq(to_eval[..., :], compute_grads=False, compute_hess=False, verbose=True, projected_diffusion=False)

        info = {'g': g, 'h': h, 'c_state': 'turn'}
        info_list.append(info)
    return info_list

def gen_constraint_data(plans_or_inits, constraint_violations, path):
    for trial in tqdm(range(1, 11)):
        info_list = []
        with open(path + f'/trial_{trial}/traj_data.p', 'rb') as f:
            data = pickle.load(f)
        if plans_or_inits == 'plans':
            data[t]['plans'] = torch.tensor(data[t]['plans']).to(device=params['device'])
            loop_range = data[t]['plans'].shape[0]
        elif plans_or_inits == 'optimizer_paths':
            loop_range = len(data[t]['optimizer_paths'])
        else:
            loop_range = len(data[t]['inits'])
        print('lr', loop_range)
        for i in tqdm(range(loop_range)):
            c_state = contact_mode_dict[data[t]['contact_state'][i].sum().item()]

            to_eval = data[t][plans_or_inits][i]
            if plans_or_inits in {'plans', 'inits'}:
                info_list = calc_info(c_state, to_eval, info_list, pregrasp_problem, index_regrasp_problem, thumb_and_middle_regrasp_problem, turn_problem, plans_or_inits)
            else:
                all_infos = []
                for csvto_tstep in tqdm(range(len(data[t]['optimizer_paths'][i]))):
                    all_infos = calc_info(c_state, to_eval[csvto_tstep], all_infos, pregrasp_problem, index_regrasp_problem, thumb_and_middle_regrasp_problem, turn_problem, plans_or_inits)
                info_list.append(all_infos)
        constraint_violations.append(info_list)

In [4]:
def get_traj(name):
    path = f'/home/abhinav/Documents/ccai/data/experiments/{name}/csvgd'

    all_data = []
    all_x = []
    # all_d2goal = []
    for trial_num in range(1, 11):
        # print(path + f'/trial_{trial_num}/trajectory.npz')
        # with np.load(path + f'/trial_{trial_num}/trajectory.npz') as data:
        #     print(data.keys)
        #     x = data['x']
        #     # d2goal = data['d2goal']
        #     all_x.append(x)
        
        with open(path + f'/trial_{trial_num}/traj_data.p', 'rb') as f:
            traj_data = pickle.load(f)
            all_data.append(traj_data)
        # all_d2goal.append(d2goal)
    constraint_violations_all = {
        # 'optimizer_paths': [],
        'plans': [],
        'inits': [],
    }
    for plans_or_inits in constraint_violations_all.keys():
        print(plans_or_inits)
        gen_constraint_data(plans_or_inits, constraint_violations_all[plans_or_inits], path)
    # Take the list of dicts and turn it into a dict of lists
    all_data = {k: [d[k] for d in all_data] for k in all_data[0]}
    all_data['violation'] = constraint_violations_all
    return all_x, all_data#, all_d2goal
data = {}

for key, name in [('rand_init_csvto', 'allegro_screwdriver_random_init_no_guidance'), ('diff_init_csvto', 'allegro_screwdriver_diffusion_init_no_guidance'), ('diff_init', 'allegro_screwdriver_diffusion_init_no_guidance_no_csvto')]:
    data[key] = {}
    all_x, all_data = get_traj(name)
    data[key]['x'] = {
        'raw': all_x,
        'mean': np.mean(all_x, axis=0),
        'std': np.std(all_x, axis=0),
    }
    data[key] = {**data[key], **all_data}
    # data[key]['d2goal'] = {
    #     'raw': all_d2goal,
    #     'mean': np.mean(all_d2goal, axis=0),
    #     'std': np.std(all_d2goal, axis=0),
    # }

# data['proj_diff_init'] = get_data('allegro_screwdriver_projected_diffusion_init_no_guidance_no_csvto')
# data['proj_diff_init_csvto'] = get_data('allegro_screwdriver_projected_diffusion_init_no_guidance')

plans


  0%|          | 0/10 [00:00<?, ?it/s]

lr 11


/home/abhinav/Documents/git_packages/pytorch_kinematics/src/pytorch_kinematics/chain.py:579: UserWarning: Using torch.cross without specifying the dim arg is deprecated.
Please either pass the dim explicitly or simply use torch.linalg.cross.
The default value of dim will change to agree with that of linalg.cross in a future release. (Triggered internally at ../aten/src/ATen/native/Cross.cpp:62.)
  H[:, :3, i, j] = torch.cross(J[:, 3:, j], J[:, :3, i])
 10%|█         | 1/10 [00:02<00:21,  2.34s/it]

lr 11


...

 80%|████████  | 8/10 [00:19<00:04,  2.44s/it]

lr 11


 90%|█████████ | 9/10 [00:22<00:02,  2.40s/it]

lr 11


100%|██████████| 10/10 [00:24<00:00,  2.44s/it]


In [11]:
def gen_plot(constraint_violations_all, key):
    plan_cv = constraint_violations_all[key]

    keys = ['pregrasp', 'index', 'thumb_middle', 'turn']
    # violation_by_mode_g = {
    #     key: [] for key in keys
    # }

    # violation_by_mode_h = {
    #     key: [] for key in keys
    # }
    g_all = []
    h_all = []
    for trial_ind in range(len(plan_cv)):
        print([i['c_state'] for i in plan_cv[trial_ind]])
        g_trial_ind = []
        h_trial_ind = []
        for contact_mode_seq in plan_cv[trial_ind]:
            g = torch.abs(contact_mode_seq['g']).mean().cpu().item()
            h = torch.relu(contact_mode_seq['h']).mean().cpu().item()
            g_trial_ind.append(g)
            h_trial_ind.append(h)
        g_all.append(g_trial_ind)
        h_all.append(h_trial_ind)
    g_all = np.stack(g_all, axis=0)
    h_all = np.stack(h_all, axis=0)
    g_all_mean = g_all.mean(axis=0)
    h_all_mean = h_all.mean(axis=0)
    g_all_std = g_all.std(axis=0)
    h_all_std = h_all.std(axis=0)
            # mode = contact_mode_seq['c_state']

            # violation_by_mode_g[mode].append(contact_mode_seq['g'])
            # violation_by_mode_h[mode].append(contact_mode_seq['h'])

    fig, ax = plt.subplots(1, 1, figsize=(12, 12))

    # plot g_all_mean with error bars
    ax.plot(g_all_mean, label='g_all_mean')
    ax.fill_between(np.arange(g_all_mean.shape[0]), g_all_mean - g_all_std, g_all_mean + g_all_std, alpha=0.5)
    ax.plot(h_all_mean, label='h_all_mean')
    ax.fill_between(np.arange(h_all_mean.shape[0]), h_all_mean - h_all_std, h_all_mean + h_all_std, alpha=0.5)
    ax.legend()

    plt.title(f'Constraint Violation: {key}')
    plt.savefig(f'/home/abhinav/Pictures/constraint_violation_analysis/{random_or_diffusion}_init_{key}.png')

    # 4 columns, 2 rows subplots
    # fig, axs = plt.subplots(2, 4, figsize=(12, 12))

    # for ind in range(4):
    #     key = keys[ind]
    #     print(violation_by_mode_g[key])
    #     this_mode_g = torch.abs(violation_by_mode_g[key][t])
    #     print(this_mode_g.shape)

    #     this_mode_h = torch.relu(violation_by_mode_h[key][t])
    #     print(this_mode_h.shape)

In [ ]:
['inits']
gen_plot(data['rand_init_csvto']['violation'], 'inits')